__Function Definition__

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  9 12:00:31 2018

@author: Hansheng Jiang
"""
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from IPython.display import display
import scipy.linalg
from itertools import combinations
from numpy import linalg
from numpy.linalg import matrix_rank
from scipy.sparse.linalg import svds, eigs
import random
import time
from scipy.optimize import minimize
import math
import matplotlib.mlab as mlab
from scipy import integrate
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
import scipy.stats
import csv
import statistics as stats
import pandas as pd
import colorsys
from scipy.integrate import quad

from sympy import sin, cos, symbols, integrate

np.random.seed(26)

In [5]:
def lmo(beta,X,y,sigma,f):
    #return the objective funciton of the linear oprimization problem
    n = len(X)
    p = len(X[0])
    obj = 0
    for i in range(n):
        obj = obj - np.exp(-0.5*(y[i] - np.dot(X[i],beta))**2 /(sigma**2))/f[i]
    return obj  

def jacobian(beta,X,y,sigma,f):
    #return gradient of the linear objective function
    n = len(X)
    p = len(X[0])
    jac = np.zeros((p,1))
    for i in range(n):
        temp = (y[i] - np.dot(X[i],beta))**2 /(sigma**2)
        jac = jac - np.exp(-0.5*temp)*temp/f[i]/(sigma**2) * np.reshape(X[i],(p,1))
        jac = np.reshape(jac,(p,1))
    jac = np.reshape(jac, (p,))
    return jac

def hessian(beta,X,y,sigma,f):
    #return Hessian of the linear objetive function
    n = len(X)
    p = len(X[0])
    hess = np.zeros((p,p))
    for i in range(n):
        temp = (y[i] - np.dot(X[i],beta))**2 /(sigma**2)
        x1 = np.reshape(X[i],(p,1))
        x2 = np.reshape(X[i],(1,p))
        temp2 = np.matmul(x1,x2)
        hess = hess - (np.exp(-0.5*temp)*temp**2/(sigma** 4) -np.exp(-0.5*temp)/(sigma**2))/f[i]*temp2
    return hess

def sollmo(X,y,sigma,f):
    #solve linear minimization oracle
    #this is an nonconvex problem with respect to beta, the result is approximal
    #return a new supporting vector g and corresponding beta
    n = len(X)
    p = len(X[0])
    
    #sensitive to initialization!!!!
    #initialize beta0 with OLS solution or beta = 0
    beta0 = np.reshape(np.dot( np.matmul(linalg.inv(np.matmul(X.T,X)),X.T),y),(p,1)) 
    
    
    #minimize exponential sum approximately
    #nonconvex problem
    OptResult = minimize(lmo, beta0, args = (X,y,sigma,f),method = 'Powell')
    beta_sol = OptResult.x
    g = np.zeros((n,1))
    for i in range(n):
        g[i] = 1/(np.sqrt(2*np.pi)*sigma)* np.exp(-0.5*(y[i] - np.dot(X[i],beta_sol))**2 /(sigma**2))
    return g,beta_sol

def FW_FC(f,alpha,P,n):
    #solve the fully corective step using classic FW
    #warm start with f from last iteration
    #P each column of P is a candidate component
    #return new f, and f as a convex combination of columns of P with coefficients alpha
    iter = 5000
    
    k = len(P[0])
    alpha = np.append(alpha,0)
    alpha = np.reshape(alpha,(k,1))
    for t in range(1,iter):
        g = 1/f
        g = np.reshape(g,(1,n))
        s = np.argmax(np.matmul(g,P))
        gamma = 2/(t+2)
        f = (1-gamma)*f +gamma*np.reshape(P[:,s],(n,1))
        temp = np.zeros((k,1))
        temp[s] = 1
        alpha = (1-gamma)*np.reshape(alpha,(k,1))+gamma*temp
    return f,alpha

def NPMLE_FW(X,y,iter,sigma):
    '''
    Use FW algorithm to solve NPMLE problem of MLR  
    sigma is estimated before
    
    '''  
    n = len(X)
    p = len(X[0])
    
    L_rec = []
    
    #initialize beta0 and f
    beta0 = np.reshape(np.dot(np.matmul(linalg.inv(np.matmul(X.T,X)),X.T),y),(p,1)) #beta0 is OLS solution
    #beta0 = np.zeros((p,1))
    f = np.zeros((n,1))
    for i in range(n):
        f[i] = 1/(np.sqrt(2*np.pi)*sigma)* np.exp(-0.5*(y[i] - np.dot(X[i],beta0))**2 /(sigma**2))
    
    # initialize P,B
    # P active set
    # B beta's corresponding to columns of P
    P = np.zeros((n,1))
    P[:,0] = f.ravel()
    B = np.zeros((p,1))
    B[:,0] = beta0.ravel()
    
    # intialize coefficients of convex combinations
    alpha = np.array([1]) 
    
    for t in range(1,iter):
        #solve LMO
        g, beta_sol = sollmo(X,y,sigma,f)
        #print("beta_sol",beta_sol)
        g = np.reshape(g,(n,1))
        beta_sol = np.reshape(beta_sol,(p,1))
        P = np.append(P,g,axis = 1)
        B = np.append(B,beta_sol,axis = 1)
        
        #fully corrective step wrt current active set P
        f, alpha = FW_FC(f,alpha,P,n)
        
        #prune P by deleting columns corresponding to zero alpha
        P_prune = np.zeros((n,1))
        B_prune = np.zeros((p,1))
        alpha_prune = np.zeros((1,))
        flag = 0
        for i in range(len(P[0])):
            if alpha[i] > 0:
                if flag == 0:
                    P_prune[:,0] = P[:,i].ravel()
                    B_prune[:,0] = B[:,i].ravel()
                    alpha_prune[0] = alpha[i]
                    flag = 1
                else:
                    P_prune = np.append(P_prune,np.reshape(P[:,i],(n,1)), axis = 1)
                    alpha_prune = np.append(alpha_prune, alpha[i])
                    B_prune = np.append(B_prune,np.reshape(B[:,i],(p,1)),axis = 1)
        
        P = P_prune
        B = B_prune
        alpha = np.reshape(alpha_prune/np.sum(alpha_prune), (len(P[0]),1))
        
        #record the change of neg-log likelihood function
        temp = np.sum(np.log(1/f))
        L_rec.append(temp)
    return f, B, alpha, L_rec, temp


def train_error(X,y,B,alpha):
    beta_ave = np.matmul(B,alpha)
    y = np.reshape(y,(len(y),1))
    y_train = np.zeros((len(y), 1))
    for i in range(len(y)):
        y_train[i] = np.matmul(X[i],beta_ave)
    y_train = np.reshape(y_train, (len(y),1))
    return linalg.norm(y - y_train)**2/len(X)


In [6]:
def L_sigma_rec_synthetic(n,iter, b1, b2,sigma):    
    '''
    return the neg-log likelihood as a function of sigma
    synthesize its own data
    test on a list of sigma
    '''
    
    #set parameters
    p =2    #number of components (currently we only consider 2 component)
    probthre = 1e-2
    pi = 0.5 # first componrnt has probability pi
    
    sigma1 = 2*sigma  # variance of 1st component
    sigma2 = sigma      #variance of 2nd component
    
    
    # synthesize two component data
    b1 = np.reshape(b1,(2,1))
    b2 = np.reshape(b2,(2,1))

    X = np.zeros((n,2))
    y = np.zeros((n,1))
    for i in range(n):
        X[i] = np.reshape([1,np.random.uniform(-1,3)],(1,2))
        z = np.random.uniform(0,1)
        if z < pi:
            y[i] = np.dot(X[i],b1) + np.random.normal(0,sigma1)
        else:
            y[i] = np.dot(X[i],b2) + np.random.normal(0,sigma2)
    
    
    sigma_list = np.arange(sigma2/2,sigma1*2,0.2)
    
    
    L_sigma_rec = []
    for sigma_est in sigma_list:
        f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma_est)
        L_sigma_rec.append(L_final)
        
    fig = plt.figure(figsize = (6,5))
    plt.plot(L_sigma_rec);
    plt.title("final likelihood over different sigma");
    return L_sigma_rec    

In [7]:
#test function with synthetic data
def test(n,iter, b1, b2, b3,pi1,pi2,sigma,sigma_est):
    # n : number of samples
    # iter : number of iterations in Frank-Wofle method
    
    #set parameters
    p =2    #number of components (currently we only consider 2 component)
    #sigma = 0.8 # standard deviation
    threprob = 0.02
    #pi1 = 0.5 # first componrnt has probability pi
    #pi2 = 0.25
    
    
    #parameters for generating synthetic data
    sigma1 = sigma  # variance of 1st component
    sigma2 = sigma      #variance of 2nd component
    sigma3 = sigma
    
    #sigma_est is what we use for Frank-Wofle method
    
    # synthesize two component data
    b1 = np.reshape(b1,(2,1))
    b2 = np.reshape(b2,(2,1))
    b3 = np.reshape(b3,(2,1))

    X = np.zeros((n,2))
    y = np.zeros((n,1))
    
    # C denots the true class of each data point
    C = np.zeros((n,1))
    
    np.random.seed(26)
    for i in range(n):
        X[i] = np.reshape([1,np.random.uniform(-1,3)],(1,2))
        z = np.random.uniform(0,1)
        if z < pi1:
            y[i] = np.dot(X[i],b1) + np.random.normal(0,sigma1)
            C[i] = 1
        elif z < pi1 + pi2 :
            y[i] = np.dot(X[i],b2) + np.random.normal(0,sigma2)
            C[i] = 2
        else:
            y[i] = np.dot(X[i],b3) + np.random.normal(0,sigma3)
            C[i] = 3
            
            
    #test what sigma we use
    #sigma_est = np.sqrt(pi* sigma1**2 + (1-pi)*sigma2**2)
    
    #run Frank-Wofle
    f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma_est)
    
    
    beta_ave = np.matmul(B,alpha)
    print("averge beta is ", beta_ave)
    print("training error is ",train_error(X,y,B,alpha) )

    beta_ols = np.reshape(np.dot(np.matmul(linalg.inv(np.matmul(X.T,X)),X.T),y),(p,1))
    print("beta_ols",beta_ols)
    #index = np.argwhere(alpha.ravel() == np.amax(alpha.ravel()))
    
    #plot
    print("final neg log likelihood is ", L_final)
    print("number of components is", len(alpha))
    print("only components with probability at least ", threprob, " are shown below:")
    
    fig_raw = plt.figure(figsize = (8,8))
    plt.scatter(X[:,1],y,color = 'black',marker = 'o',label = 'Noisy data', facecolors = 'None');
    ax = plt.gca()
    ax.set_xlim([-2,4])
    ax.set_ylim([-3,8])
    plt.legend(loc=2, bbox_to_anchor=(0., -0.05),borderaxespad=0.);
    plt.show();
    
    fig0 = plt.figure(figsize = (8,8))
    for i in range(len(y)):
        if C[i] == 1:
            plt.scatter(X[i][1],y[i],color = 'red',marker = 'o',label = 'Class 1', facecolors = 'None');
        elif C[i] == 2:
            plt.scatter(X[i][1],y[i],color = 'blue',marker = 'o',label = 'Class 2', facecolors = 'None');
        else:
            plt.scatter(X[i][1],y[i],color = 'green',marker = 'o',label = 'Class 3', facecolors = 'None');
            
    t = np.arange(np.amin(X[:,1])-0.5,np.amax(X[:,1])+0.5,1e-6)
    #plt.plot(t,b1[0]+b1[1]*t,'r',t,b2[0]+b2[1]*t,'red')
    i = 0
    plt.plot(t,b1[0]+b1[1]*t, color = 'red',linewidth = pi1*8 )
    plt.plot(t,b2[0]+b2[1]*t, color = 'blue',linewidth = pi2*8 )
    if pi1 + pi2 < 1:
        plt.plot(t,b3[0]+b3[1]*t, color = 'green',linewidth = (1-pi2-pi2)*8 )
           
            
    #plt.plot(t,beta_ols[0]+beta_ols[1]*t,'green')  
    if pi1 + pi2 <1:
        custom_lines = [(Line2D([], [], color='red', marker='o',markerfacecolor = 'None', linestyle='None',linewidth = 8*pi1),Line2D([], [], color='red')),
                        (Line2D([], [], color='blue', marker='o',markerfacecolor = 'None', linestyle='None',linewidth = 8*pi2),Line2D([], [], color='blue')),
                         (Line2D([], [], color='green', marker='o',markerfacecolor = 'None', linestyle='None',linewidth = 8*(1-pi1-pi2)),Line2D([], [], color='green'))
                        #Line2D([0], [0], color= 'red'# ),
                        #Line2D([0], [0], color='black')
                        #,Line2D([0], [0], color='green')#
                        ]
        plt.legend(custom_lines, ['y = %.1f + %.1f x with probility %.2f' %(b1[0], b1[1], pi1), #,'True mixture'# 
                                  'y = %.1f + %.1f x with probility %.2f' %(b2[0], b2[1], pi2),
                                  'y = %.1f + %.1f x with probility %.2f' %(b3[0], b3[1], 1-pi1-pi2),
                                   #'NPMLE component'#, 'OLS'#
                                 ],loc = 2,bbox_to_anchor=(0., -0.05),borderaxespad=0.);
    else:
        custom_lines = [(Line2D([], [], color='red', marker='o',markerfacecolor = 'None', linestyle='None',linewidth = 8*pi1),Line2D([], [], color='red')),
                        (Line2D([], [], color='blue', marker='o',markerfacecolor = 'None', linestyle='None',linewidth = 8*pi2),Line2D([], [], color='blue')),
                    
                        #Line2D([0], [0], color= 'red'# ),
                        #Line2D([0], [0], color='black')
                        #,Line2D([0], [0], color='green')#
                        ]
        plt.legend(custom_lines, ['y = %.1f + %.1f x with probility %.2f' %(b1[0], b1[1], pi1), #,'True mixture'# 
                                  'y = %.1f + %.1f x with probility %.2f' %(b2[0], b2[1], pi2),
                                
                                   #'NPMLE component'#, 'OLS'#
                                 ],loc=2,bbox_to_anchor=(0., -0.05),borderaxespad=0.);
    ax = plt.gca()
    ax.set_xlim([-2,4])
    ax.set_ylim([-3,8])
    plt.show();

    
    
    fig1 = plt.figure(figsize = (8,8))

    t = np.arange(np.amin(X[:,1])-0.5,np.amax(X[:,1])+0.5,1e-6)
    
    
    #plt.plot(t,b1[0]+b1[1]*t,'r',t,b2[0]+b2[1]*t,'red')
    
    N = len(alpha)
    
    RGB_tuples = [(240,163,255),(0,117,220),(153,63,0),(76,0,92),(0,92,49),
    (43,206,72),(255,204,153),(128,128,128),(148,255,181),(143,124,0),(157,204,0),
    (194,0,136),(0,51,128),(255,164,5),(255,168,187),(66,102,0),(255,0,16),(94,241,242),(0,153,143),
    ( 224,255,102),(116,10,255),(153,0,0),(255,255,128),(255,255,0),(25,25,25),(255,80,5)]
    
    component_plot = []
    component_color = []
    
    temp = 0
    index_sort = np.argsort(-np.reshape(alpha,(len(alpha),)))
    for i in index_sort:
        b = B[:,i]
        if alpha[i] >threprob:
            component_plot.append(i)
            component_color.append(temp)
            plt.plot(t,b[0]+b[1]*t, color = tuple( np.array(RGB_tuples[temp])/255),linewidth = alpha[i][0]*8 ,label = 'y = %.4f + %.4f x with probility %.2f' %(b[0], b[1], alpha[i]))
            temp = temp + 1
            print("coefficients", b, "with probability", alpha[i])
    
    # we ONLY do clustering based on plotted components, i.e. components with high probability (>threprob)
    C_cluster = np.zeros((n,1))
    for i in range(len(y)):
        prob = np.zeros((N,1))
        for j in component_plot:
            prob[j] = alpha[j] * np.exp(-0.5*(y[i] - np.dot(X[i],B[:,j]))**2 /(sigma**2))
        C_cluster[i] = np.argmax(prob)
        plt.scatter(X[i][1],y[i],color = tuple(np.array(RGB_tuples[component_color[component_plot.index(C_cluster[i])]])/255) ,marker = 'o', facecolors = 'None'); 
            
    #plt.plot(t,beta_ols[0]+beta_ols[1]*t,'green')  
    
    #custom_lines = [Line2D([], [], color='blue', marker='o',markerfacecolor = 'None', linestyle='None'),
                    #Line2D([0], [0], color= 'red'# ),
                    #Line2D([0], [0], color='black')
                    #,Line2D([0], [0], color='green')#
                    #,]
    #plt.legend(custom_lines, ['Noisy data'#,'True mixture'# 
                            #  , 'NPMLE component'#, 'OLS'#
                            # ],loc=0);
    plt.legend(bbox_to_anchor=(0., -0.05), loc=2, borderaxespad=0.)
    ax = plt.gca()
    ax.set_xlim([-2,4])
    ax.set_ylim([-3,8])
    plt.show();

    
    #plot Hellinger distance
    fig2 = plt.figure(figsize = (20,3))
    x_list = [-3,-1,1,3,5] #List of x values
    i = 0
    
    for i in range(len(x_list)):
        x = x_list[i]
        y = np.linspace(- 15, 15, 100)
           
        #calculate difference of suqre root of density functions
        dist_fit = lambda y: (np.sqrt(0.5*scipy.stats.norm.pdf(y-(b1[0]+b1[1]*x), 0, sigma)+0.5*scipy.stats.norm.pdf(y-(b1[0]+b1[1]*x),0, sigma)) \
        - np.sqrt(sum(alpha[i]*scipy.stats.norm.pdf( y - (B[0,i]+B[1,i]*x), 0, sigma) for i in range(len(alpha)))))**2
        
        print("Fix x = %.1f, squared Hellinger distance for NPMLE is %.5f" % (x, quad(dist_fit, -np.inf, np.inf)[0]))

        
        plt.subplot(1,len(x_list),i+1)
        plt.plot(y,pi1*scipy.stats.norm.pdf(y - (b1[0]+b1[1]*x), 0, sigma)+pi2*scipy.stats.norm.pdf(y-(b2[0]+b2[1]*x),0, sigma)+(1-pi1-pi2)*scipy.stats.norm.pdf(y-(b3[0]+b3[1]*x),0, sigma),'red',label = 'True distribution')
        plt.plot(y, sum(alpha[i]*scipy.stats.norm.pdf( y-(B[0,i]+B[1,i]*x), 0, sigma) for i in range(len(alpha))),'black',label = 'NPMLE distribution')
        plt.title("x = %.1f"%x)
    custom_lines = [
                Line2D([0], [0], color= 'red'),
                Line2D([0], [0], color='black'),
                #Line2D([0], [0], color='green')#
        ]
    plt.legend(custom_lines, ['True mixture', 'Fitted mixture'#, 'OLS'#
                             ], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    
    fig3 = plt.figure(figsize = (6,5))
    plt.plot(L_rec);plt.title("neg-log likelihood over iterations");
      
    

__Synthetic Data__

In [ ]:
#colorful
test(5000,50,(1,1),(4,-1),(-1,0.5),0.5,0.5,sigma = 0.8,sigma_est=0.8)

In [ ]:
#colorful
test(300,50,(0,1),(5,1),(0,1),0.5,0.5,sigma = 1,sigma_est = 1)

In [ ]:
#colorful
test(500,50,(1,1.5),(3,-1),(-1,0.5),0.3,0.3,sigma = 0.5,sigma_est = 0.5)

__Sigma Choice__

In [ ]:
n = 150
p =2    #number of components (currently we only consider 2 component)
sigma = 0.2 # standard deviation
probthre = 1e-2
pi = 0.5 # first componrnt has probability pi

sigma1 = sigma  # variance of 1st component
sigma2 = sigma      #variance of 2nd component

# synthesize two component data
b1 = np.reshape([1,1],(2,1))
b2 = np.reshape([4,-1],(2,1))

X = np.zeros((n,2))
y = np.zeros((n,1))
for i in range(n):
    X[i] = np.reshape([1,np.random.uniform(-1,3)],(1,2))
    z = np.random.uniform(0,1)
    if z < pi:
        y[i] = np.dot(X[i],b1) + np.random.normal(0,sigma1)
    else:
        y[i] = np.dot(X[i],b2) + np.random.normal(0,sigma2)


In [ ]:
sigma_max = np.sqrt(stats.variance(np.reshape(y, (len(y),))))
sigma_min = 0.1
sigma_list = np.sqrt(np.arange(sigma_min**2, sigma_max**2, 0.06**2))
number = len(X)
#random permutation to shuffle data
index = np.random.permutation(number)

X_shuffled = X[index]
y_shuffled = y[index]

In [ ]:
L_sigma_rec_synthetic(500,300, (1,1), (4,-1),0.6)

__5 fold cross validation__

In [ ]:
L_sigma_rec,train_error_sigma_rec, test_error_sigma_rec = L_sigma(X_shuffled[:120],y_shuffled[:120],X_shuffled[120:150],y_shuffled[120:150],sigma_list)
L_sigma_rec2,train_error_sigma_rec2, test_error_sigma_rec2 = L_sigma(X_shuffled[30:150],y_shuffled[30:150],X_shuffled[:30],y_shuffled[:30],sigma_list)
L_sigma_rec3,train_error_sigma_rec3, test_error_sigma_rec3 = L_sigma(np.append(X_shuffled[:30], X_shuffled[60:150],axis = 0),np.append(y_shuffled[:30], y_shuffled[60:150],axis = 0),X_shuffled[30:60],y_shuffled[30:60],sigma_list)
L_sigma_rec4,train_error_sigma_rec4, test_error_sigma_rec4 = L_sigma(np.append(X_shuffled[:60], X_shuffled[90:150],axis = 0),np.append(y_shuffled[:60], y_shuffled[90:150],axis = 0),X_shuffled[60:90],y_shuffled[60:90],sigma_list)
L_sigma_rec5,train_error_sigma_rec5, test_error_sigma_rec5 = L_sigma(np.append(X_shuffled[:90], X_shuffled[120:150],axis = 0),np.append(y_shuffled[:90], y_shuffled[120:150],axis = 0),X_shuffled[90:120],y_shuffled[90:120],sigma_list)
L_sigma_rec_CV = np.zeros((len(sigma_list),1))
L_sigma_rec_CV = np.array(L_sigma_rec)+np.array(L_sigma_rec2)+ np.array(L_sigma_rec3)+ np.array(L_sigma_rec4)+ np.array(L_sigma_rec5)
train_error_sigma_rec_CV = np.zeros((len(sigma_list),1))
train_error_sigma_rec_CV = np.array(train_error_sigma_rec) + np.array(train_error_sigma_rec2) + np.array(train_error_sigma_rec3) + np.array(train_error_sigma_rec4) + np.array(train_error_sigma_rec5)
train_error_sigma_rec_CV = train_error_sigma_rec_CV/5
fig1 = plt.figure(figsize = (6,5))
plt.plot(sigma_list, L_sigma_rec_CV);plt.title("5-fold CV neg-log likelihood");
    
fig2 = plt.figure(figsize = (6,5))
plt.plot(sigma_list, train_error_sigma_rec_CV);plt.title("5-fold CV taining error");
    
fig3 = plt.figure(figsize = (6,5))
plt.plot(sigma_list, test_error_sigma_rec_CV);plt.title("5-fold CV validation error");

__Hellinger risk scaling wrt n__

In [8]:
#test function with synthetic data
def hellinger_risk(n,num,iter, b1, b2, b3,pi1,pi2,sigma,sigma_est):
    '''
    Return the average risk on #num random experiments
    
    '''

    hellinger_rec = []
    
    for j in range(num):
        
        p =2    #number of components (currently we only consider 2 component)
        #sigma = 0.8 # standard deviation
        threprob = 0.02
        #pi1 = 0.5 # first componrnt has probability pi
        #pi2 = 0.25


        #parameters for generating synthetic data
        sigma1 = sigma  # variance of 1st component
        sigma2 = sigma      #variance of 2nd component
        sigma3 = sigma

        #sigma_est is what we use for Frank-Wofle method

        # synthesize two component data
        b1 = np.reshape(b1,(2,1))
        b2 = np.reshape(b2,(2,1))
        b3 = np.reshape(b3,(2,1))

        X = np.zeros((n,2))
        y = np.zeros((n,1))

        # C denots the true class of each data point
        C = np.zeros((n,1))

        for i in range(n):
            X[i] = np.reshape([1,np.random.uniform(-1,3)],(1,2))
            z = np.random.uniform(0,1)
            if z < pi1:
                y[i] = np.dot(X[i],b1) + np.random.normal(0,sigma1)
                C[i] = 1
            elif z < pi1 + pi2 :
                y[i] = np.dot(X[i],b2) + np.random.normal(0,sigma2)
                C[i] = 2
            else:
                y[i] = np.dot(X[i],b3) + np.random.normal(0,sigma3)
                C[i] = 3

        f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma_est)
        hellinger = 0
        for i in range(n):
            x = X[i][1]           

            #calculate difference of suqre root of density functions
            dist_fit = lambda y: (np.sqrt(0.5*scipy.stats.norm.pdf(y-(b1[0]+b1[1]*x), 0, sigma)+0.5*scipy.stats.norm.pdf(y-(b1[0]+b1[1]*x),0, sigma)) \
            - np.sqrt(sum(alpha[i]*scipy.stats.norm.pdf( y - (B[0,i]+B[1,i]*x), 0, sigma) for i in range(len(alpha)))))**2

            hellinger = hellinger + quad(dist_fit, -np.inf, np.inf)[0]
        hellinger_rec.append(hellinger/n)
    return hellinger_rec
        

In [ ]:
hellinger_rec = hellinger_risk(50,20,30,(1,1),(4,-1),(-1,0.5),0.5,0.5,sigma = 0.8,sigma_est=0.8)

In [10]:
num = 20

In [ ]:
hellinger_n_rec = []
for n in np.arange(5,5000,50):
    hellinger_n_rec.append(np.sum(hellinger_risk(50,num,20,(1,1),(10,-1),(-1,0.5),0.5,0.5,sigma = 0.3,sigma_est=0.3))/num)

/global/software/sl-7.x86_64/modules/langs/python/3.5/lib/python3.5/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)


In [ ]:
z = np.arange(5,5000,50)
np.divide(z,np.log(z)**(3.5))


In [ ]:
np.divide(z,np.log(z)**(3.5))

In [ ]:
hellinger_n_rec2 = []
for n in np.arange(500,1000,50):
    hellinger_n_rec2.append(np.sum(hellinger_risk(50,num,20,(1,1),(4,-1),(-1,0.5),0.5,0.5,sigma = 0.3,sigma_est=0.3))/num)

__Continuous Measure__

In [ ]:
y = 1
x = 0
sigma = 0.3
meanb1 = [1,1]
covb1 = [[0.01,0],[0,0.2]]
meanb2 = [5,2]
covb2 = [[0.01,0],[0,0.2]]

from scipy import integrate

# density function under continuous measure
def func_xy(x,y):
    func = lambda b,k: scipy.stats.norm.pdf(y - (b+k*x), 0, sigma)*(0.5*scipy.stats.norm.pdf(b, meanb1[0], np.sqrt(covb1[0][0]))*\
                                                            scipy.stats.norm.pdf(k, meanb1[1], np.sqrt(covb1[1][1]))+ 0.5*scipy.stats.norm.pdf(b, meanb2[0], np.sqrt(covb2[0][0]))\
                                                           *scipy.stats.norm.pdf(k, meanb2[1], np.sqrt(covb2[1][1])))
    return func


In [ ]:
#test function with synthetic data
def test_continuous(n,iter):
    # n : number of samples
    # iter : number of iterations in Frank-Wofle method
    
    #set parameters
    p =2    #number of components (currently we only consider 2 component)
    sigma = 0.3 # standard deviation
    threprob = 0.02   
    
    #sigma_est is what we use for Frank-Wofle method
    #one guess is to use the std of y
    sigma_est = sigma
    
    X = np.zeros((n,2))
    y = np.zeros((n,1))
    
    np.random.seed(26)
    for i in range(n):
        b_pi = np.random.uniform(0,1)
        
        X[i] = np.reshape([1,np.random.uniform(-1,3)],(1,2))
        meanb1 = [1,1]
        covb1 = [[0.01,0],[0,0.2]]
        meanb2 = [5,2]
        covb2 = [[0.01,0],[0,0.2]]
        b1 = np.reshape(np.random.multivariate_normal(meanb1,covb1,1),(2,1))
        b2 = np.reshape(np.random.multivariate_normal(meanb2,covb2,1),(2,1))

        '''
    
        b1 = np.random.beta(2,6,size = (2,1))
        b2 = np.array([[1],[0.5]])+np.random.beta(2,2,size = (2,1))
        '''
        if b_pi < 0.5:
            b = b1
        else:
            b = b2
        y[i] = np.dot(X[i],b) + np.random.normal(0,sigma)
    
    
    #run Frank-Wofle
    f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma_est)
    
    
    beta_ave = np.matmul(B,alpha)
    print("averge beta is ", beta_ave)
    print("training error is ",train_error(X,y,B,alpha) )

    beta_ols = np.reshape(np.dot(np.matmul(linalg.inv(np.matmul(X.T,X)),X.T),y),(p,1))
    print("beta_ols",beta_ols)
    #index = np.argwhere(alpha.ravel() == np.amax(alpha.ravel()))
    
    #plot
    print("final neg log likelihood is ", L_final)
    print("number of components is", len(alpha))
    print("only components with probability at least ", threprob, " are shown below:")
    
    fig_raw = plt.figure(figsize = (8,8))
    for i in range(len(y)):
         plt.scatter(X[i][1],y[i],color = 'black',marker = 'o', facecolors = 'None');
    
    
    fig1 = plt.figure(figsize = (8,8))

    t = np.arange(np.amin(X[:,1])-0.5,np.amax(X[:,1])+0.5,1e-6)
    
    
    #plt.plot(t,b1[0]+b1[1]*t,'r',t,b2[0]+b2[1]*t,'red')
    
    N = len(alpha)
    
    RGB_tuples = [(240,163,255),(0,117,220),(153,63,0),(76,0,92),(0,92,49),
    (43,206,72),(255,204,153),(128,128,128),(148,255,181),(143,124,0),(157,204,0),
    (194,0,136),(0,51,128),(255,164,5),(255,168,187),(66,102,0),(255,0,16),(94,241,242),(0,153,143),
    ( 224,255,102),(116,10,255),(153,0,0),(255,255,128),(255,255,0),(25,25,25),(255,80,5)]
    
    component_plot = []
    component_color = []
    
    temp = 0
    index_sorted = np.argsort(-np.reshape(alpha,(len(alpha),)))
    for i in index_sorted:
        b = B[:,i]
        if alpha[i] >threprob:
            component_plot.append(i)
            component_color.append(temp)
            plt.plot(t,b[0]+b[1]*t, color = tuple( np.array(RGB_tuples[temp])/255),linewidth = alpha[i][0]*8,label = 'y = %7f + %.7f x with prob %.2f' %(b[0], b[1], alpha[i]) )
            temp = temp + 1
            print("coefficients", b, "with probability", alpha[i])
    
    # we ONLY do clustering based on plotted components, i.e. components with high probability (>threprob)
    C_cluster = np.zeros((n,1))
    for i in range(len(y)):
        prob = np.zeros((N,1))
        for j in component_plot:
            prob[j] = alpha[j] * np.exp(-0.5*(y[i] - np.dot(X[i],B[:,j]))**2 /(sigma**2))
        C_cluster[i] = np.argmax(prob)
        plt.scatter(X[i][1],y[i],color = tuple(np.array(RGB_tuples[component_color[component_plot.index(C_cluster[i])]])/255) ,marker = 'o', facecolors = 'None'); 
    plt.legend(bbox_to_anchor=(1.05, 1), loc=9, borderaxespad=0.)     
    plt.show();
    
    #plot Hellinger distance
    fig2 = plt.figure(figsize = (20,3))
    x_list = [-3,-1,1,3,5] #List of x values
    i = 0
    
    for i in range(len(x_list)):
        x = x_list[i]
        y = np.linspace(- 15, 15, 100)
        z = np.zeros(len(np.linspace(- 15, 15, 100)))
        for j in range(len(z)):
            z[j] = integrate.dblquad(func_xy(x,y[j]),-np.inf,np.inf,-np.inf,np.inf)[0]
        plt.subplot(1,len(x_list),i+1)
        plt.plot(y,z,'red',label = 'True distribution')
        plt.plot(y, sum(alpha[i]*scipy.stats.norm.pdf( y-(B[0,i]+B[1,i]*x), 0, sigma) for i in range(len(alpha))),'black',label = 'NPMLE distribution')
        #plt.plot(y, scipy.stats.norm.pdf(y-(beta_ols[0]+beta_ols[1]*x), 0, sigma),'green',label = 'OLS distribution')  
        #plt.xlabel('y')
        #plt.ylabel('pdf')
        plt.title("x = %.1f"%x)
    custom_lines = [
                Line2D([0], [0], color= 'red'),
                Line2D([0], [0], color='black'),
                #Line2D([0], [0], color='green')#
        ]
    plt.legend(custom_lines, ['True mixture', 'Fitted mixture'#, 'OLS'#
                             ], loc = (1.05,1))
    plt.show()
    
    fig3 = plt.figure(figsize = (6,5))
    plt.plot(L_rec);plt.title("neg-log likelihood over iterations");
      
    

In [ ]:
test_continuous(500,30)

__Tone perception data__

In [ ]:
#read tonedata into file
with open('tonedata.csv', newline='') as csvfile:
    data = np.array(list(csv.reader(csvfile)))
    data = data[1:]
dataf = data.astype(np.float)
n = np.shape(dataf)[0]
ones = np.ones((n,1))
X = np.concatenate((ones, np.reshape(dataf[:,1],(n,1))), axis = 1)
y = np.reshape(dataf[:,2],(n,1))

In [ ]:
iter = 200
threprob = 0.02

#Use Frank-Wofle with an estimated sigma
sigma = 0.03
f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma)


#beta_ols = np.reshape(np.dot(np.matmul(linalg.inv(np.matmul(X.T,X)),X.T),y),(p,1))

In [ ]:
#plot
print("final neg log likelihood is ", L_final)
print("number of components is", len(alpha))
print("only components with probability at least ", threprob, " are shown below:")

fig1 = plt.figure(figsize = (8,8))
plt.scatter(X[:,1],y,color = 'black',marker = 'o',label = 'Noisy data', facecolors = 'None');
t = np.arange(np.amin(X[:,1])-0.5,np.amax(X[:,1])+0.5,1e-6)
#plt.plot(t,b1[0]+b1[1]*t,'r',t,b2[0]+b2[1]*t,'red')
i = 0

index_sorted = np.argsort(-np.reshape(alpha,(len(alpha),)))
for i in index_sorted:
    b = B[:,i]
    if alpha[i] >threprob:
        plt.plot(t,b[0]+b[1]*t, color = str((1-alpha[i][0])/100),linewidth = alpha[i][0]*8 ,label = 'y = %.4f + %.4f x with prob %.2f' %(b[0], b[1], alpha[i]) )
        print("coefficients", b, "with probability", alpha[i])
#plt.plot(t,beta_ols[0]+beta_ols[1]*t,'green')  
#plt.legend(custom_lines, ['Noisy data'#,'True mixture'# 
                         # , 'NPMLE component'#, 'OLS'#
                         #],loc=2);
plt.legend(loc=9, bbox_to_anchor=(1.05, 1),borderaxespad=0.) 
plt.show();

In [ ]:
#define a range of candidate sigma values
sigma_max = np.sqrt(stats.variance(np.reshape(y, (len(y),))))
sigma_min = 0.06
sigma_list = np.sqrt(np.arange(sigma_min**2, sigma_max**2, sigma_min**2))

In [ ]:
def L_sigma(X,y,X_test,y_test,sigma_list):
    '''
    test X,y
    '''
    L_sigma_rec = []
    train_error_sigma_rec = []
    test_error_sigma_rec = []
        
    iter = 200
    threprob = 1e-2
    y = np.reshape(y,(len(y),1))
    y_test = np.reshape(y_test,(len(y_test),1))
    
    for sigma in sigma_list:
        f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma)
        L_sigma_rec.append(L_final)
        
        beta_ave = np.matmul(B,alpha)
        
        y_train = np.zeros((len(y), 1))
        for i in range(len(y)):
            y_train[i] = np.matmul(X[i],beta_ave)
        y_train = np.reshape(y_train, (len(y),1))
        train_error_sigma_rec.append(linalg.norm(y_train - y)**2/len(X))
        
        y_test_pred = np.zeros((len(y_test), 1))
        for i in range(len(y_test)):
            y_test_pred[i] = np.matmul(X[i],beta_ave)
        y_test_pred = np.reshape(y_test_pred, (len(y_test),1))
        test_error_sigma_rec.append(linalg.norm(y_test - y_test_pred)**2/len(X))
    
    fig1 = plt.figure(figsize = (6,5))
    plt.plot(sigma_list, L_sigma_rec);
    
    fig2 = plt.figure(figsize = (6,5))
    plt.plot(sigma_list, train_error_sigma_rec);
    
    fig3 = plt.figure(figsize = (6,5))
    plt.plot(sigma_list, test_error_sigma_rec)
    
    return L_sigma_rec,train_error_sigma_rec, test_error_sigma_rec

In [ ]:
#define a range of candidate sigma values
sigma_max = np.sqrt(stats.variance(np.reshape(y, (len(y),))))
sigma_min = 0.03
sigma_list = np.sqrt(np.arange(sigma_min**2, sigma_max**2, 0.06**2))

In [ ]:
subset = np.random.choice(len(y), 10)
L_sigma_rec = L_sigma(X[subset],y[subset],sigma_list)

In [ ]:
number = len(X)
#random permutation to shuffle data
index = np.random.permutation(number)

X_shuffled = X[index]
y_shuffled = y[index]

In [ ]:
L_sigma_rec,train_error_sigma_rec, test_error_sigma_rec = L_sigma(X_shuffled[:40],y_shuffled[:40],X_shuffled[40:50],y_shuffled[40:50],sigma_list)

__5-Fold Cross Validation for Tone Data__

In [ ]:
L_sigma_rec,train_error_sigma_rec, test_error_sigma_rec = L_sigma(X_shuffled[:120],y_shuffled[:120],X_shuffled[120:150],y_shuffled[120:150],sigma_list)

In [ ]:
L_sigma_rec2,train_error_sigma_rec2, test_error_sigma_rec2 = L_sigma(X_shuffled[30:150],y_shuffled[30:150],X_shuffled[:30],y_shuffled[:30],sigma_list)

In [ ]:
L_sigma_rec3,train_error_sigma_rec3, test_error_sigma_rec3 = L_sigma(np.append(X_shuffled[:30], X_shuffled[60:150],axis = 0),np.append(y_shuffled[:30], y_shuffled[60:150],axis = 0),X_shuffled[30:60],y_shuffled[30:60],sigma_list)

In [ ]:
L_sigma_rec4,train_error_sigma_rec4, test_error_sigma_rec4 = L_sigma(np.append(X_shuffled[:60], X_shuffled[90:150],axis = 0),np.append(y_shuffled[:60], y_shuffled[90:150],axis = 0),X_shuffled[60:90],y_shuffled[60:90],sigma_list)

In [ ]:
L_sigma_rec5,train_error_sigma_rec5, test_error_sigma_rec5 = L_sigma(np.append(X_shuffled[:90], X_shuffled[120:150],axis = 0),np.append(y_shuffled[:90], y_shuffled[120:150],axis = 0),X_shuffled[90:120],y_shuffled[90:120],sigma_list)

In [ ]:
L_sigma_rec_CV = np.zeros((len(sigma_list),1))
L_sigma_rec_CV = np.array(L_sigma_rec)+np.array(L_sigma_rec2)+ np.array(L_sigma_rec3)+ np.array(L_sigma_rec4)+ np.array(L_sigma_rec5)

In [ ]:
train_error_sigma_rec_CV = np.zeros((len(sigma_list),1))
train_error_sigma_rec_CV = np.array(train_error_sigma_rec) + np.array(train_error_sigma_rec2) + np.array(train_error_sigma_rec3) + np.array(train_error_sigma_rec4) + np.array(train_error_sigma_rec5)
train_error_sigma_rec_CV = train_error_sigma_rec_CV/5

In [ ]:
fig1 = plt.figure(figsize = (6,5))
plt.plot(sigma_list, L_sigma_rec_CV);plt.title("5-fold CV neg-log likelihood");
    
fig2 = plt.figure(figsize = (6,5))
plt.plot(sigma_list, train_error_sigma_rec_CV);plt.title("5-fold CV taining error");
    
fig3 = plt.figure(figsize = (6,5))
plt.plot(sigma_list, test_error_sigma_rec_CV);plt.title("5-fold CV validation error");

__GDP-CO2 Data__

In [ ]:
#read gdp-co2data into file
with open('gdp-co2.csv', newline='') as csvfile:
    data = np.array(list(csv.reader(csvfile)))
dataf = data[:,1:4].astype(np.float)
n = np.shape(dataf)[0]
ones = np.ones((n,1))
X = np.concatenate((ones, np.reshape(dataf[:,1]/dataf[:,0]*100,(n,1))), axis = 1)
y = np.reshape(dataf[:,2]/dataf[:,0]*1e+6,(n,1))

In [ ]:
iter = 200
threprob = 1e-2

#Use Frank-Wofle with an estimated sigma
sigma = 2
f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma)

##########IMPORTANT subproblem initializes with beta = 0


In [ ]:
#plot
print("final neg log likelihood is ", L_final)
print("number of components is", len(alpha))
print("only components with probability at least ", threprob, " are shown below:")

fig1 = plt.figure(figsize = (8,8))
plt.scatter(X[:,1],y,color = 'blue',marker = 'o',label = 'Noisy data', facecolors = 'None');
t = np.arange(np.amin(X[:,1])-0.5,np.amax(X[:,1])+0.5,1e-2)
i = 0
for i in range(len(alpha)):
    b = B[:,i]
    if alpha[i] >threprob:
        plt.plot(t,b[0]+b[1]*t, color = str((1-alpha[i][0])/100),linewidth = alpha[i][0]*8 )
        print("coefficients", b, "with probability", alpha[i]) 
custom_lines = [Line2D([], [], color='blue', marker='o',markerfacecolor = 'None', linestyle='None'),
                Line2D([0], [0], color='black')
                ,]
plt.legend(custom_lines, ['Noisy data'
                          , 'NPMLE component'
                         ],loc=9);
plt.show();

__Aphids Data__

In [ ]:
#read aphids into file
with open('aphids.csv', newline='') as csvfile:
    data = np.array(list(csv.reader(csvfile)))
dataf = data[1:,1:3].astype(np.float)
n = np.shape(dataf)[0]
ones = np.ones((n,1))
X = np.concatenate((ones, np.reshape(dataf[:,0],(n,1))), axis = 1)
y = np.reshape(dataf[:,1],(n,1))

In [ ]:
def test_aphids(sigma):
    iter = 200
    #Use Frank-Wofle with an estimated sigma
    #sigma = 3.3
    f, B, alpha, L_rec, L_final = NPMLE_FW(X,y,iter,sigma)
    threprob = 0.02
    #plot
    print("final neg log likelihood is ", L_final)
    print("number of components is", len(alpha))
    print("only components with probability at least ", threprob, " are shown below:")

    fig1 = plt.figure(figsize = (8,8))
    plt.scatter(X[:,1],y,color = 'blue',marker = 'o',label = 'Noisy data', facecolors = 'None');
    t = np.arange(np.amin(X[:,1])-0.5,np.amax(X[:,1])+0.5,1e-2)
    i = 0
    index_sorted = np.argsort(-np.reshape(alpha,(len(alpha),)))
    for i in index_sorted:
        b = B[:,i]
        if alpha[i] >threprob:
            plt.plot(t,b[0]+b[1]*t, color = str((1-alpha[i][0])/100),linewidth = alpha[i][0]*8 ,label = 'y = %.4f + %.4f x with prob %.2f' %(b[0], b[1], alpha[i]) )
            print("coefficients", b, "with probability", alpha[i]) 
    custom_lines = [Line2D([], [], color='blue', marker='o',markerfacecolor = 'None', linestyle='None'),
                    Line2D([0], [0], color='black')
                    ,]
    #plt.legend(custom_lines, ['Noisy data'
                 #             , 'NPMLE component'
                  #           ],loc=0);
    plt.legend(loc=9, bbox_to_anchor=(1.05, 1),borderaxespad=0.) 
    plt.show();

In [ ]:
for sigma in np.arange(1,3,0.1):
    print("**************************",sigma)
    test_aphids(sigma)